In [ ]:
!pip install BeautifulSoup4
!pip install sqlalchemy psycopg2-binary pandas
!pip install requests


In [ ]:
from urllib.request import urlopen
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup as soup
import json
import time
import requests
from bs4 import BeautifulSoup
import requests
import json
import time

In [ ]:
def scrape_books(base_url="http://books.toscrape.com/", timeout=10, wait=1, save_json=True):

    url = base_url + "catalogue/page-1.html"
    libros = []
    i = 1

    while url:
        try:
            print(f"\nAccediendo a: {url}")

            uClient = urlopen(url, timeout=timeout)
            page_html = uClient.read()
            uClient.close()

            page_soup = soup(page_html, "html.parser")

            bookshelf = page_soup.findAll("li", {"class": "col-xs-6 col-sm-4 col-md-3 col-lg-3"})

            for book in bookshelf:
                book_title = book.h3.a["title"]
                a_href = book.h3.a["href"]
                book_url = base_url + "catalogue/" + a_href.replace("../../../", "")

                libros.append({
                    "titulo": book_title,
                    "url": book_url
                })

                print(f"{i}. {book_title}")
                i += 1

            next_button = page_soup.find("li", {"class": "next"})
            if next_button:
                next_href = next_button.a["href"]
                url = base_url + "catalogue/" + next_href
            else:
                url = None

            time.sleep(wait)

        except HTTPError as e:
            print(f"Error HTTP {e.code} al acceder a {url}")
            break
        except URLError as e:
            print(f"Error de conexión: {e.reason}")
            break
        except Exception as e:
            print(f"Error inesperado: {e}")
            break

    if save_json:
        with open("libros_url.json", "w", encoding="utf-8") as f:
            json.dump(libros, f, ensure_ascii=False, indent=4)

    print(f"\nTotal de libros scrapeados: {len(libros)}")
    return libros


In [ ]:
# Llamar a la función
libros = scrape_books()

In [ ]:
url = []
for libro in libros:
    url.append(libro["url"])

In [ ]:

def obtener_datos_libro(url_libro):
    response = requests.get(url_libro)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Título
    titulo = soup.h1.text.strip()

    # Precio
    precio_tag = soup.find('p', class_='price_color')
    precio = precio_tag.text.strip()

    # Rating
    rating_tag = soup.find('p', class_='star-rating')
    rating_str = rating_tag['class'][1]

    rating_map = {
        "One": 1,
        "Two": 2,
        "Three": 3,
        "Four": 4,
        "Five": 5
    }

    rating = rating_map.get(rating_str, 0) 

    # Descripción
    descripcion_tag = soup.select_one('#product_description ~ p')
    descripcion = descripcion_tag.text.strip() if descripcion_tag else ""

    # Género (3er <li> en el breadcrumb)
    breadcrumb = soup.select("ul.breadcrumb li a")
    genero = breadcrumb[2].text.strip() if len(breadcrumb) > 2 else ""


    # Tabla de datos adicionales
    tabla = soup.find('table', class_='table table-striped')
    filas = tabla.find_all('tr')
    datos = {fila.th.text.strip(): fila.td.text.strip() for fila in filas}

    # Diccionario con los datos
    libro_data = {
        "Titulo": titulo,
        "Precio": precio,
        "rating": rating,
        "descripcion": descripcion,
        "Genero": genero, 
        "Upc": datos.get('UPC', ''),
        "tipo_producto": datos.get('Product Type', ''),
        "precio_excl_tax": datos.get('Price (excl. tax)', ''),
        "precio_incl_tax": datos.get('Price (incl. tax)', ''),
        "tax": datos.get('Tax', ''),
        "numero_stock": ''.join(filter(str.isdigit, datos.get('Availability', ''))),
        "numero_reviews": datos.get('Number of reviews', ''),
    }

    return libro_data

# Procesar todos los libros y guardar al final
todos_los_libros = []
i = 1
for libro in libros:
    print(f"Libro scrapeado {i}")
    datos = obtener_datos_libro(libro["url"])
    todos_los_libros.append(datos)
    i += 1

# Guardar todo en un único JSON
with open("libros.json", "w", encoding="utf-8") as f:
    json.dump(todos_los_libros, f, ensure_ascii=False, indent=4)

print(f"\n✅ Total de libros guardados: {len(todos_los_libros)}")

In [ ]:
def obtener_autores_google_books(archivo_in="libros.json", archivo_out="autores_libros.json"):
    with open(archivo_in, "r", encoding="utf-8") as f:
        libros = json.load(f)

    autores_data = []

    for libro in libros:
        query = libro["Titulo"].replace(" ", "+")  # limpiar búsqueda
        url = f"https://www.googleapis.com/books/v1/volumes?q=intitle:{query}"

        try:
            resp = requests.get(url, timeout=10).json()

            autores = []
            if "items" in resp:
                volumen = resp["items"][0]["volumeInfo"]
                autores = volumen.get("authors", [])

            autores_data.append({
                "titulo": libro["Titulo"],
                "autores": autores
            })

            print(f"{libro['Titulo']} → {autores if autores else 'No encontrado'}")

            time.sleep(0.5)  # anti-rate-limit

        except Exception as e:
            print(f"Error con '{libro['Titulo']}': {e}")
            autores_data.append({
                "titulo": libro["Titulo"],
                "autores": []
            })

    with open(archivo_out, "w", encoding="utf-8") as f:
        json.dump(autores_data, f, ensure_ascii=False, indent=4)

    return autores_data
obtener_autores_google_books()

In [5]:
import sqlite3

def crear_bd_completa():
    conn = sqlite3.connect("biblioteca.db")
    cursor = conn.cursor()
    cursor.execute("PRAGMA foreign_keys = ON;")

    # Tabla de géneros
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS generos (
        id_genero INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre TEXT UNIQUE NOT NULL
    );
    """)

    # Tabla de autores
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS autores (
        id_autor INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre TEXT UNIQUE NOT NULL
    );
    """)

    # Tabla de libros con todos los campos scrapeados
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS libros (
        id_libro INTEGER PRIMARY KEY AUTOINCREMENT,
        titulo TEXT NOT NULL,
        precio DECIMAL NOT NULL,
        rating INTEGER NOT NULL,
        descripcion TEXT NOT NULL,
        upc TEXT NOT NULL,
        tipo_producto TEXT NOT NULL,
        precio_excl_tax DECIMAL NOT NULL,
        precio_incl_tax DECIMAL NOT NULL,
        tax DECIMAL NOT NULL,
        numero_stock INTEGER NOT NULL,
        numero_reviews INTEGER NOT NULL
    );
    """)

    # Relación N:N entre libros y autores
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS libro_autor (
        id_libro INTEGER NOT NULL,
        id_autor INTEGER NOT NULL,
        PRIMARY KEY (id_libro, id_autor),
        FOREIGN KEY (id_libro) REFERENCES libros(id_libro) ON DELETE CASCADE,
        FOREIGN KEY (id_autor) REFERENCES autores(id_autor) ON DELETE CASCADE
    );
    """)

    # Relación N:N entre libros y géneros
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS libro_genero (
        id_libro INTEGER NOT NULL,
        id_genero INTEGER NOT NULL,
        PRIMARY KEY (id_libro, id_genero),
        FOREIGN KEY (id_libro) REFERENCES libros(id_libro) ON DELETE CASCADE,
        FOREIGN KEY (id_genero) REFERENCES generos(id_genero) ON DELETE CASCADE
    );
    """)

    conn.commit()
    conn.close()
    print("Base de datos completa creada con libros, autores, géneros y relaciones N:N.")

crear_bd_completa()


Base de datos completa creada con libros, autores, géneros y relaciones N:N.


In [6]:
import sqlite3
import json

def cargar_generos(json_file="libros.json", db_file="biblioteca.db"):
    # Abrir JSON
    with open(json_file, "r", encoding="utf-8") as f:
        libros = json.load(f)

    # Extraer géneros únicos
    generos_unicos = set()
    for libro in libros:
        genero = libro.get("Genero")
        if genero:
            generos_unicos.add(genero.strip())

    # Conectar a la base de datos
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Insertar géneros en la tabla
    for genero in generos_unicos:
        try:
            cursor.execute("INSERT OR IGNORE INTO generos (nombre) VALUES (?)", (genero,))
        except Exception as e:
            print(f"Error al insertar {genero}: {e}")

    conn.commit()
    conn.close()
    print(f"Se cargaron {len(generos_unicos)} géneros en la tabla 'generos'.")
cargar_generos()


Se cargaron 50 géneros en la tabla 'generos'.


In [7]:
def cargar_autores(autores_file="autores_libros.json", db_file="biblioteca.db"):
    with open(autores_file, "r", encoding="utf-8") as f:
        autores_json = json.load(f)

    autores_unicos = set()
    for item in autores_json:
        for autor in item.get("autores", []):
            autores_unicos.add(autor.strip())

    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    for autor in autores_unicos:
        cursor.execute("INSERT OR IGNORE INTO autores (nombre) VALUES (?)", (autor,))

    conn.commit()
    conn.close()
    print(f"{len(autores_unicos)} autores cargados")

cargar_autores()

932 autores cargados


In [8]:
import sqlite3
import json

def insertar_libros(libros_file="libros.json", db_file="biblioteca.db"):
    with open(libros_file, "r", encoding="utf-8") as f:
        libros = json.load(f)

    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    for libro in libros:
        try:
            cursor.execute("""
                INSERT INTO libros (
                    titulo, precio, rating, descripcion, upc, tipo_producto,
                    precio_excl_tax, precio_incl_tax, tax, numero_stock, numero_reviews
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (
                libro["Titulo"],
                float(libro.get("Precio", 0)),
                int(libro.get("rating", 0)),
                libro.get("descripcion", ""),
                libro.get("Upc", ""),
                libro.get("tipo_producto", ""),
                float(libro.get("precio_excl_tax", 0)),
                float(libro.get("precio_incl_tax", 0)),
                float(libro.get("tax", 0)),
                int(libro.get("numero_stock", 0)),
                int(libro.get("numero_reviews", 0))
            ))
        except Exception as e:
            print(f"Error al insertar {libro.get('Titulo', 'sin titulo')}: {e}")

    conn.commit()
    conn.close()
    print(f"Se intentaron insertar {len(libros)} libros.")

insertar_libros()


Error al insertar A Light in the Attic: could not convert string to float: '£51.77'
Error al insertar Tipping the Velvet: could not convert string to float: '£53.74'
Error al insertar Soumission: could not convert string to float: '£50.10'
Error al insertar Sharp Objects: could not convert string to float: '£47.82'
Error al insertar Sapiens: A Brief History of Humankind: could not convert string to float: '£54.23'
Error al insertar The Requiem Red: could not convert string to float: '£22.65'
Error al insertar The Dirty Little Secrets of Getting Your Dream Job: could not convert string to float: '£33.34'
Error al insertar The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull: could not convert string to float: '£17.93'
Error al insertar The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics: could not convert string to float: '£22.60'
Error al insertar The Black Maria: could not convert string to float: '£52.15'
E

In [ ]:
def relacion_libro_genero(libros_file="libros.json", db_file="biblioteca.db"):
    with open(libros_file, "r", encoding="utf-8") as f:
        libros = json.load(f)

    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    for libro in libros:
        genero = libro.get("Genero", "").strip()
        if genero:
            cursor.execute("SELECT id_genero FROM generos WHERE nombre = ?", (genero,))
            res = cursor.fetchone()
            if res:
                id_genero = res[0]
                cursor.execute("SELECT id_libro FROM libros WHERE titulo = ?", (libro["Titulo"],))
                id_libro = cursor.fetchone()[0]
                cursor.execute("INSERT OR IGNORE INTO libro_genero (id_libro, id_genero) VALUES (?, ?)", (id_libro, id_genero))

    conn.commit()
    conn.close()
    print("Relaciones libro ↔ género creadas.")

relacion_libro_genero()

In [ ]:
def relacion_libro_autor(libros_file="libros.json", autores_file="autores_libros.json", db_file="biblioteca.db"):
    with open(libros_file, "r", encoding="utf-8") as f:
        libros = json.load(f)
    with open(autores_file, "r", encoding="utf-8") as f:
        autores_json = json.load(f)

    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    for libro in libros:
        cursor.execute("SELECT id_libro FROM libros WHERE titulo = ?", (libro["Titulo"],))
        id_libro = cursor.fetchone()[0]

        autores_libro = next((item["autores"] for item in autores_json if item["titulo"] == libro["Titulo"]), [])
        for autor in autores_libro:
            cursor.execute("SELECT id_autor FROM autores WHERE nombre = ?", (autor,))
            res = cursor.fetchone()
            if res:
                id_autor = res[0]
                cursor.execute("INSERT OR IGNORE INTO libro_autor (id_libro, id_autor) VALUES (?, ?)", (id_libro, id_autor))

    conn.commit()
    conn.close()
    print("Relaciones libro ↔ autor creadas.")
    relacion_libro_autor()

In [9]:
!pip install nb-mermaid

%%javascript
require(["nb-mermaid"], function(nbmermaid) {
nbmermaid.renderMermaid();
});

SyntaxError: invalid syntax (2657407470.py, line 4)

# Diagrama UML de la Base de Datos

```mermaid
erDiagram
    LIBRO ||--o{ LIBRO_AUTOR : tiene
    AUTOR ||--o{ LIBRO_AUTOR : pertenece_a
    
    LIBRO ||--o{ LIBRO_GENERO : tiene
    GENERO ||--o{ LIBRO_GENERO : pertenece_a

    LIBRO {
        int id_libro PK
        string titulo
        float precio
        int rating
        string descripcion
        string upc
        string desctipo_producto
        decimal precio_excl_tax
        decimal precio_incl_tax
        decimal tax
        int numero_stock
        int numero_reviews
        int autor_id FK
        int genero_id FK
    }

    AUTOR {
        int id_autor PK
        string nombre
    }

    GENERO {
        int id_genero PK
        string nombre
    }

    LIBRO_AUTOR {
        int id_libro FK
        int id_autor FK
    }

    LIBRO_GENERO {
        int id_libro FK
        int id_genero FK
    }



In [ ]:
print(1)